# Optimización de Límites de Crédito

## Exploración de Datos

### 1. - CARGA DE LIBRERIAS

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

ModuleNotFoundError: No module named 'pandas'

### 2. -Carga de los Datos

In [ ]:
#file_path_training = os.path.abspath('../data/train-data/data_training.csv')
#file_path_test = os.path.abspath('../data/test-data/data_training.csv')


In [ ]:
# read training and test dataset
df_training = pd.read_csv('../data/train-data/data_training.csv', sep=';', index_col='ID', na_values='#N/D')
df_test = pd.read_csv('../data/test-data/data_test.csv', sep=';', index_col='ID', na_values='#N/D')

# display the first few rows of the training dataset
df_training.head()

In [ ]:
# Analizar todas las columnas
for column in df_training.columns:
  print(f"Column name: {column}")
  print(f"Data type: {df_training[column].dtype}")
  print(f"Number of unique values: {df_training[column].nunique()}")
  print(f"Number of missing values: {df_training[column].isnull().sum()}")
  print(f"Sample values: {df_training[column].head(3)}")
  print("-" * 60)

### Conclusiones:
1.- Se Elimina las columnas que no aportan Informacion para el estudio: Age, Years_employed

In [ ]:
df_training.drop(columns = ['Age','Years_employed'], inplace = True)
df_test.drop(columns = ['Age','Years_employed'], inplace = True)

In [ ]:
#Realizamos un análisis de nulos para validar si existen datos faltantes
df_training.isna().sum().sort_values(ascending = False)
df_test.isna().sum().sort_values(ascending = False)

Conclusion: Todo bien. No existen datos nulos.

In [ ]:
#Función de variables categóricas
def graficos_eda_categoricos(cat):
    
    #Calculamos el número de filas que necesitamos
    from math import ceil
    filas = ceil(cat.shape[1] / 2)

    #Definimos el gráfico
    f, ax = plt.subplots(nrows = filas, ncols = 2, figsize = (16, filas * 6))

    #Aplanamos para iterar por el gráfico como si fuera de 1 dimensión en lugar de 2
    ax = ax.flat 

    #Creamos el bucle que va añadiendo gráficos
    for cada, variable in enumerate(cat):
        cat[variable].value_counts().plot.barh(ax = ax[cada])
        ax[cada].set_title(variable, fontsize = 12, fontweight = "bold")
        ax[cada].tick_params(labelsize = 12)

In [ ]:
#Ejecutamos la función de variables categóricas.
graficos_eda_categoricos(df_training.select_dtypes('O'))

In [ ]:
# convert data to table
table_training = pa.Table.from_pandas(df_training)
table_test = pa.Table.from_pandas(df_test)

In [ ]:
# write tables out to parquet
pq.write_table(table_training, "../data/train-data/data-training.parquet", version="1.0")
pq.write_table(table_test, "../data/test-data/data-test.parquet", version="1.0")

Connect to your workspace

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)